In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import os
import sys
root_folders = ['..', '../..', './']
root_folders = [os.path.abspath(os.path.join(x)) for x in root_folders]
for path in root_folders:
    if path not in sys.path:
        sys.path.append(path)
import nltk
%load_ext autoreload
%autoreload 2

from deeplavrov.deeplavrov.vocabulary.vocabulary import WordVocabEncoder


Using TensorFlow backend.


In [2]:
encoder = WordVocabEncoder.load('anki_index.json')

In [3]:
text = 'Tom likes Mary, but Mary likes John. To make matters even more interesting, John likes Alice, but Alice likes Tom.'

In [4]:
indices = encoder.to_indices(text)

In [5]:
indices

[1,
 64,
 837,
 900,
 82,
 4378,
 900,
 837,
 3941,
 3,
 3394,
 1356,
 688,
 1135,
 909,
 2973,
 82,
 3941,
 837,
 9951,
 82,
 4378,
 9951,
 837,
 64,
 3,
 4]

In [6]:
encoder.to_text(indices)

'Tom likes Mary , but Mary likes John . To make matters even more interesting , John likes Alice , but Alice likes Tom .'

In [7]:
pad_indices = encoder.pad_indices(indices)

In [8]:
pad_indices

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    1,   64,  837,  900,   82, 4378,
        900,  837, 3941,    3, 3394, 1356,  688, 1135,  909, 2973,   82,
       3941,  837, 9951,   82, 4378, 9951,  837,   64,    3,    4])

In [9]:
encoder.to_text(pad_indices)

'Tom likes Mary , but Mary likes John . To make matters even more interesting , John likes Alice , but Alice likes Tom .'

Из текста в последовательности с padding:

In [10]:
encoder.text_to_indices(text)

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    1,   64,  837,  900,   82, 4378,
        900,  837, 3941,    3, 3394, 1356,  688, 1135,  909, 2973,   82,
       3941,  837, 9951,   82, 4378, 9951,  837,   64,    3,    4])

Пример с генератором

In [26]:
def generator():
    with open('ru_anki_corpora.txt', 'r', encoding='utf-8') as f, open('eng_anki_corpora.txt', 'r', encoding='utf-8') as d:
        ru_line, eng_line = f.readline().strip(), d.readline().strip()
        while ru_line:
            yield encoder.text_to_indices(eng_line, True), encoder.text_to_indices(ru_line,False)
            ru_line, eng_line = f.readline().strip(), d.readline().strip()

In [30]:
dataset = tf.data.Dataset.from_generator(generator=generator,
                                         output_types=(tf.int64, tf.int64))

In [31]:
dataset = dataset.batch(2, drop_remainder=True)

In [33]:
for (x,y) in dataset:
    x, y = x.numpy(), y.numpy()
    x = [encoder.to_text(seq) for seq in x]
    y = [encoder.to_text(seq, is_source=False) for seq in y]
    for ru, eng in zip(x,y):
        print(ru, eng)
    break

Go . Иди .
Go . Идите .


In [ ]:
tf.__version__